In [1]:
import pandas as pd
import os
import numpy as np
import json
import tarfile
import time

In [2]:
# use this command to compress the uncompressed tarfile when the scrape is ready:
# xz -zk tarfile.tar

In [3]:
DATE = "2023_01_08"
TYPE = "_diff"

In [4]:
tar = tarfile.open(f"../databases/archives/{DATE}_rawdata{TYPE}.tar", "r")

In [5]:
def create_nutrition_df(data, name):
    transformed_data = [name] + [row[2] for row in data["data"]]
    columns = ["id"] + [row[0] for row in data["data"]]
    return pd.DataFrame([transformed_data], columns=columns)

In [6]:
start_time = time.time()
all_nutrition_df = []
for member in tar.getmembers():
    if member.name.find("nutrition.json") != -1:
        file_content = tar.extractfile(member.name).read().decode("utf-8")
        data = json.loads(file_content)
        all_nutrition_df.append(create_nutrition_df(data, member.name.split("/")[-2]))

nutritions_df = pd.concat(all_nutrition_df, ignore_index=True)
end_time = time.time()
print(end_time - start_time)
print((end_time - start_time) / len(nutritions_df))

5.1681067943573
0.03856796115192015


In [7]:
nutritions_df

,id,Energia (kJ / kcal),Zsír (g),Telített zsírsavak (g),Szénhidrát (g),Cukrok (g),Fehérje (g),Só (g),Rost (g),Folsav (µg),...,Magnézium (mg),Vas (mg),Cink (mg),Réz (mg),Mangán (mg),Fluor (µg),Szelén (µg),Króm (µg),Molibdén (µg),Jód (µg)
0,3687481,100.000000/,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3719638,100.000000/,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3687478,100.000000/,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3685162,630.000000/152.000000,10.000000,0.800000,10.000000,7.000000,2.000000,0.700000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,36463,2359/566,35.0000,15.8000,58.0000,25.0000,4.4000,0.1000,0.1000,200.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,4834,226/54,0.1800,0.0500,11.5000,9.9000,0.4900,0.0100,1.9800,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,3924492,2279/546,32.0000,18.0000,58.0000,49.0000,5.1000,0.3900,1.7000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,4061362,1538/366,14,"7,5",52,28,"6,8","0,46","2,6",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,4118347,2480/593,43,"20,5",48,47,2,"0,1",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
nutritions_df.to_csv(f"../databases/csv/{DATE}/nutritions_raw.csv", escapechar="\\")

In [9]:
def create_df(data: dict):
    return pd.DataFrame([data.values()], columns=list(data.keys()))

In [10]:
start_time = time.time()
all_product_df = []
all_variant_df = []
all_prices_df = []
for i, member in enumerate(tar.getmembers()):
    if member.name.find("product.json") != -1:
        file_content = tar.extractfile(member.name).read().decode("utf-8")
        data = json.loads(file_content)

        price_data = data["selectedVariant"]["price"]
        price_data["id"] = data["selectedVariant"]["id"]
        all_prices_df.append(create_df(price_data))

        del data["selectedVariant"]["price"]
        del data["selectedVariant"]["packageInfo"]["unitPrice"]
        data["selectedVariant"]["packageUnit"] = data["selectedVariant"]["packageInfo"]["packageUnit"]
        data["selectedVariant"]["packageSize"] = data["selectedVariant"]["packageInfo"]["packageSize"]
        del data["selectedVariant"]["packageInfo"]

        all_variant_df.append(create_df(data["selectedVariant"]))

        data["selectedVariant"] = data["selectedVariant"]["id"]
        data["defaultVariant"] = data["defaultVariant"]["id"]
        all_product_df.append(create_df(data))

    if i % 1000 == 0:
        print(f'processed {i} members in {time.time() - start_time} seconds')

df_products = pd.concat(all_product_df, ignore_index=True)
df_variants = pd.concat(all_variant_df, ignore_index=True)
df_prices = pd.concat(all_prices_df, ignore_index=True)
end_time = time.time()
print(end_time - start_time)
print((end_time - start_time) / len(df_products))

processed 0 members in 0.007473945617675781 seconds
processed 1000 members in 9.354341983795166 seconds
processed 2000 members in 16.872031688690186 seconds
processed 3000 members in 23.92822265625 seconds
processed 4000 members in 30.922845125198364 seconds
processed 5000 members in 36.9918794631958 seconds
processed 6000 members in 43.4241418838501 seconds
processed 7000 members in 49.465006589889526 seconds
processed 8000 members in 55.39689612388611 seconds
processed 9000 members in 61.554136753082275 seconds
processed 10000 members in 67.64376926422119 seconds
processed 11000 members in 74.16762924194336 seconds
processed 12000 members in 80.14476275444031 seconds
processed 13000 members in 86.21012902259827 seconds
processed 14000 members in 92.3974494934082 seconds
processed 15000 members in 98.68774127960205 seconds
processed 16000 members in 105.17986488342285 seconds
processed 17000 members in 111.81332659721375 seconds
processed 18000 members in 118.94082522392273 seconds
pr

In [11]:
df_prices.head(10)

,net,gross,currency,decimalPlaces,netDiscounted,grossDiscounted,discountPercentage,isDiscounted,id,discountValidFrom,discountValidTo
0,529,529,HUF,0,529,529,0,False,55371,NaN,NaN
1,549,549,HUF,0,549,549,0,False,4026669,NaN,NaN
2,549,549,HUF,0,549,549,0,False,4026672,NaN,NaN
3,549,549,HUF,0,549,549,0,False,4026675,NaN,NaN
4,549,549,HUF,0,549,549,0,False,4026678,NaN,NaN
5,579,579,HUF,0,579,579,0,False,9898,NaN,NaN
6,579,579,HUF,0,579,579,0,False,54825,NaN,NaN
7,579,579,HUF,0,579,579,0,False,54828,NaN,NaN
8,689,689,HUF,0,689,689,0,False,28354,NaN,NaN
9,689,689,HUF,0,689,689,0,False,28357,NaN,NaN


In [12]:
df_variants.drop_duplicates(subset=["id"], inplace=True)
df_products.drop_duplicates(subset=["id"], inplace=True)
df_prices.drop_duplicates(subset=["id"], inplace=True)
df_variants

,id,name,sku,productId,addedName,selectValue,status,unit,eanCode,aided,...,flags,media,details,isInVirtualStock,shoppingListsContain,offerType,packageUnit,packageSize,roll,itemVolumeInfo
0,55371,Friskies Steril száraz macskaeledel lazaccal é...,541547,55032,,55371,none,db,7613033000201,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"[description, ingredients]",False,[],,KG,0.300,NaN,NaN
1,4026669,PreVital teljes értékű állateledel felnőtt mac...,465410,489276,,4026669,none,db,5999566111235,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"[description, ingredients]",False,[],,KG,0.300,NaN,NaN
2,4026672,PreVital teljes értékű állateledel felnőtt mac...,465411,489279,,4026672,none,db,5999566111228,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"[description, ingredients]",False,[],,KG,0.300,NaN,NaN
3,4026675,PreVital teljes értékű állateledel ivartalanít...,465412,489282,,4026675,none,db,5999566111259,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"[description, ingredients]",False,[],,KG,0.300,NaN,NaN
4,4026678,PreVital teljes értékű állateledel lakásban va...,465413,489285,,4026678,none,db,5999566111242,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"[description, ingredients]",False,[],,KG,0.300,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98205,3998149,Milka szaloncukor joghurtízű krémmel 310 g,411578,460765,,3998149,none,db,7622201125325,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"[description, ingredients, nutrition, allergens]",False,[],,KG,0.310,NaN,NaN
98206,3998137,Milka szaloncukor válogatás 310 g,411573,460753,,3998137,none,db,7622201125363,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"[description, ingredients, nutrition, allergens]",False,[],,KG,0.310,NaN,NaN
98209,3998122,Oreo vaníliaízű töltelékkel töltött kakaós kek...,411565,460738,,3998122,none,db,7622300491215,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"[description, ingredients, parameterList, nutr...",False,[],,KG,0.246,NaN,NaN
98211,46400,Stühmer ír krémlikőr ízű szaloncukor 340 g,752237,46328,,46400,none,db,5999565690366,False,...,"[{'flag': 'flag_domestic_processed', 'name': '...",{'images': ['https://ahuazurewebblob0.azureedg...,"[description, ingredients, nutrition, allergens]",False,[],,KG,0.350,NaN,NaN


In [13]:
assert len(df_products) == len(df_variants) == len(df_prices)

In [14]:
df_variants.to_csv(f"../databases/csv/{DATE}/variants_raw.csv", escapechar="\\", compression='xz')
df_products.to_csv(f"../databases/csv/{DATE}/products_raw.csv", escapechar="\\", compression='xz')
df_prices.to_csv(f"../databases/csv/{DATE}/prices_raw.csv", escapechar="\\", compression='xz')

In [15]:
df_variants.loc[df_variants["details"].str.contains("allergens").fillna(False)]

,id,name,sku,productId,addedName,selectValue,status,unit,eanCode,aided,...,flags,media,details,isInVirtualStock,shoppingListsContain,offerType,packageUnit,packageSize,roll,itemVolumeInfo


In [16]:
df_variants["details"].str[2].str.contains("allergens")

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
         ...  
98205    False
98206    False
98209    False
98211    False
98213    False
Name: details, Length: 32302, dtype: object

In [17]:
df_variants.loc[df_variants["details"].str[-1] == "allergens", "id"]

17         31879
18       4039889
22         27982
28       4038599
30         11428
          ...   
98205    3998149
98206    3998137
98209    3998122
98211      46400
98213    4060729
Name: id, Length: 7825, dtype: int64

In [18]:
df_variants = pd.read_csv(f"../databases/csv/{DATE}/variants_raw.csv", index_col=0, compression='xz')
df_products = pd.read_csv(f"../databases/csv/{DATE}/products_raw.csv", index_col=0, compression='xz')
df_prices = pd.read_csv(f"../databases/csv/{DATE}/prices_raw.csv", index_col=0, compression='xz')

In [19]:
df_products

,id,categoryId,categoryName,brandName,defaultVariant,selectedVariant,eancode,usedItem,reviewable,reviewSum,categories,inCategories,isNewProduct,stockInfos,adultsOnly,shipmentDays,ageConfirmed,isNonFood,documents,extraWeightPrice
0,55032,11909,Száraz macskaeledel,FRISKIES,55371,55371,7613033000201,"{'canReturn': False, 'return': False}",login,"{'sumCount': 0, 'average': 0}","[{'id': 12617, 'name': 'Állateledel, Otthon, H...",[11909],False,[],False,0,False,False,[],NaN
1,489276,11909,Száraz macskaeledel,PreVital,4026669,4026669,5999566111235,"{'canReturn': False, 'return': False}",login,"{'sumCount': 0, 'average': 0}","[{'id': 12617, 'name': 'Állateledel, Otthon, H...",[11909],False,[],False,0,False,False,[],NaN
2,489279,11909,Száraz macskaeledel,PreVital,4026672,4026672,5999566111228,"{'canReturn': False, 'return': False}",login,"{'sumCount': 0, 'average': 0}","[{'id': 12617, 'name': 'Állateledel, Otthon, H...",[11909],False,[],False,0,False,False,[],NaN
3,489282,11909,Száraz macskaeledel,PreVital,4026675,4026675,5999566111259,"{'canReturn': False, 'return': False}",login,"{'sumCount': 0, 'average': 0}","[{'id': 12617, 'name': 'Állateledel, Otthon, H...",[11909],False,[],False,0,False,False,[],NaN
4,489285,11909,Száraz macskaeledel,PreVital,4026678,4026678,5999566111242,"{'canReturn': False, 'return': False}",login,"{'sumCount': 0, 'average': 0}","[{'id': 12617, 'name': 'Állateledel, Otthon, H...",[11909],False,[],False,0,False,False,[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98205,460765,6837,NaN,Milka,3998149,3998149,7622201125325,"{'canReturn': False, 'return': False}",login,"{'sumCount': 0, 'average': 0}",[],[14227],False,[],False,1,False,False,[],NaN
98206,460753,6837,NaN,Milka,3998137,3998137,7622201125363,"{'canReturn': False, 'return': False}",login,"{'sumCount': 0, 'average': 0}",[],[14227],False,[],False,1,False,False,[],NaN
98209,460738,5791,NaN,Milka,3998122,3998122,7622300491215,"{'canReturn': False, 'return': False}",login,"{'sumCount': 0, 'average': 0}",[],[14227],False,[],False,1,False,False,[],NaN
98211,46328,6837,NaN,STÜHMER,46400,46400,5999565690366,"{'canReturn': False, 'return': False}",login,"{'sumCount': 0, 'average': 0}",[],[14227],False,[],False,1,False,False,[],NaN


In [20]:
df_variants

,id,name,sku,productId,addedName,selectValue,status,unit,eanCode,aided,...,flags,media,details,isInVirtualStock,shoppingListsContain,offerType,packageUnit,packageSize,roll,itemVolumeInfo
0,55371,Friskies Steril száraz macskaeledel lazaccal é...,541547,55032,NaN,55371,none,db,7613033000201,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"['description', 'ingredients']",False,[],NaN,KG,0.300,NaN,NaN
1,4026669,PreVital teljes értékű állateledel felnőtt mac...,465410,489276,NaN,4026669,none,db,5999566111235,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"['description', 'ingredients']",False,[],NaN,KG,0.300,NaN,NaN
2,4026672,PreVital teljes értékű állateledel felnőtt mac...,465411,489279,NaN,4026672,none,db,5999566111228,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"['description', 'ingredients']",False,[],NaN,KG,0.300,NaN,NaN
3,4026675,PreVital teljes értékű állateledel ivartalanít...,465412,489282,NaN,4026675,none,db,5999566111259,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"['description', 'ingredients']",False,[],NaN,KG,0.300,NaN,NaN
4,4026678,PreVital teljes értékű állateledel lakásban va...,465413,489285,NaN,4026678,none,db,5999566111242,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"['description', 'ingredients']",False,[],NaN,KG,0.300,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98205,3998149,Milka szaloncukor joghurtízű krémmel 310 g,411578,460765,NaN,3998149,none,db,7622201125325,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"['description', 'ingredients', 'nutrition', 'a...",False,[],NaN,KG,0.310,NaN,NaN
98206,3998137,Milka szaloncukor válogatás 310 g,411573,460753,NaN,3998137,none,db,7622201125363,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"['description', 'ingredients', 'nutrition', 'a...",False,[],NaN,KG,0.310,NaN,NaN
98209,3998122,Oreo vaníliaízű töltelékkel töltött kakaós kek...,411565,460738,NaN,3998122,none,db,7622300491215,False,...,[],{'images': ['https://ahuazurewebblob0.azureedg...,"['description', 'ingredients', 'parameterList'...",False,[],NaN,KG,0.246,NaN,NaN
98211,46400,Stühmer ír krémlikőr ízű szaloncukor 340 g,752237,46328,NaN,46400,none,db,5999565690366,False,...,"[{'flag': 'flag_domestic_processed', 'name': '...",{'images': ['https://ahuazurewebblob0.azureedg...,"['description', 'ingredients', 'nutrition', 'a...",False,[],NaN,KG,0.350,NaN,NaN


In [21]:
df_nutritions = pd.read_csv("../databases/csv/2022_12_19/nutritions_raw.csv", index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '../databases/csv/2022-12-19/nutritions_raw.csv'

In [ ]:
df_nutritions

In [ ]:
# merge variants with prices and nutritions
df_merged = df_variants.merge(df_prices, on="id")
df_merged = df_merged.merge(df_nutritions, on="id")

In [ ]:
df_merged.sort_values(by="discountPercentage", ascending=False).head(100)[["name", "gross", "discountPercentage", "grossDiscounted"]]

In [ ]:
df_merged.loc[df_merged["name"].str.contains("zab"), ["name", "Fehérje (g)"]]

# Collect categories

In [ ]:
start_time = time.time()
all_categories_df = []
for member in tar.getmembers():
    if member.name.find("product.json") != -1:
        file_content = tar.extractfile(member.name).read().decode("utf-8")
        data = json.loads(file_content)
        all_categories_df.append(pd.DataFrame(data["categories"]))


df_categories = pd.concat(all_categories_df, ignore_index=True)
end_time = time.time()
print(end_time - start_time)
print((end_time - start_time) / len(df_products))

In [ ]:
df_categories["id"].value_counts()

In [ ]:
df_categories.drop_duplicates(subset=["id"], inplace=True)

In [ ]:
df_categories = df_categories.sort_values(by="id").reset_index(drop=True)

In [ ]:
df_categories

In [ ]:
df_categories.to_csv(f"../databases/csv/{DATE}categories_raw.csv", escapechar="\\")